In [ ]:
import joblib
import tarfile
import os, shutil

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

fname = 'data.csv'
model_name = 'model'
label = 'LABEL'

In [ ]:
df = pd.read_csv(fname)
df.fillna('', inplace=True)

### Preparing the data


In [ ]:
test_size = 0.3
epochs = 5
batch_size = 32

Create test and train sets. Split into data + labels. 

In [ ]:
y = data[label]
x = data.drop(columns=[label])

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_size) 

print('Train: {}'.format(x_train.shape))
print('Test: {}'.format(x_test.shape))

## Create Pipeline

In [ ]:
def save_pipeline(pipeline, name='model'):      
        
    def make_tarfile(output_filename, source_dir):
        with tarfile.open(output_filename, "w:gz") as tar:
            tar.add(source_dir, arcname=os.path.basename(source_dir))

    
    os.makedirs('temp', exist_ok=True)
    joblib.dump(pipeline.named_steps['encoder'], 'temp/encoder.joblib')
    joblib.dump(pipeline.named_steps['classifier'].classes_, 'temp/classes.joblib')
    pipeline.named_steps['classifier'].model.save('temp/model.h5')
    make_tarfile(name+'.tar.gz', 'temp')
    shutil.rmtree('temp')
    
    
def load_pipeline(name='model'):
    
    tf = tarfile.open(name+'.tar.gz')
    tf.extractall()
    encoder = joblib.load('temp/encoder.joblib')
    build_model = lambda: load_model('temp/model.h5')
    classifier = KerasClassifier(build_fn=build_model, verbose=1)
    classifier.classes_ = joblib.load('temp/classes.joblib')
    classifier.model = build_model()
    

    pipeline = Pipeline(steps=[
      ('encoder', encoder),
      ('classifier', classifier)
    ])

    shutil.rmtree('temp')

    return pipeline
    

def create_model(init='normal', act='relu', l='binary_crossentropy', opt='adam', m=['accuracy']):
    model = Sequential()
    model.add(Dense(100, kernel_initializer=init, activation=act))
    model.add(Dense(1, kernel_initializer=init, activation='sigmoid'))

    model.compile(loss=l, optimizer=opt, metrics=m)

    return model


In [ ]:
encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
classifier = KerasClassifier(build_fn=create_model, verbose=1, epochs=epochs, batch_size=batch_size)

In [ ]:
pipeline = Pipeline(steps=[
  ('encoder', encoder),
  ('classifier', classifier)
])

In [ ]:
pipeline.fit(x_train, y_train)

In [ ]:
save_pipeline(pipeline, model_name)

In [ ]:
p = load_pipeline(model_name)

## Test the pipeline

In [ ]:
predicted = p.predict(x_test)

In [ ]:
actual = list(np.array(y_test))

In [ ]:
acc = accuracy_score(actual, predicted)
report = classification_report(actual, predicted, digits=3)
cf = confusion_matrix(actual, predicted)

In [ ]:
print(report)
print("Accuracy", acc)

In [ ]:
sns.heatmap(cf, xticklabels=['No' + label, label], yticklabels=['No' + label, label], annot=True, fmt="d")
plt.title("Confusion matrix")
plt.ylabel('True Class')
plt.xlabel('Predicted class')
plt.show()